In [1]:
import os
import numpy as np

In [ ]:
src = 'data/train'

texts = []
labels = []

for label in ['neg', 'pos']:
    dir_name = os.path.join(src label)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f